In [6]:
# do basic imports and unpack McMurdo data

from pmagpy import ipmag
reload(ipmag)
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
reload(data_model3)
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

import pmagpy.controlled_vocabularies3 as cv



/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df
got full_df
got parsed_df


In [18]:
dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
con = Contribution(dir_path)

loc_dm = con.tables['locations'].data_model.dm['locations']
loc_df = con.tables['locations'].df
cols = loc_df.columns

current_df = loc_df
current_dm = loc_dm
# replace NaN with None
current_dm = current_dm.where((pd.notnull(current_dm)), None)
cols = current_df.columns


#vocab.get_controlled_vocabularies()

current_df.loc['Tel Hazor']['lat_s'] = 400.
current_df

-W- No such file: /Users/nebula/Python/PmagPy/3_0/Megiddo/images.txt


,citations,lat_n,lat_s,location,location_type,lon_e,lon_w
location,,,,,,,
Tel Hazor,This study,33.0168,400,Tel Hazor,Archeological Site,35.568,35.568
Tel Megiddo,This study,32.585,32.585,Tel Megiddo,Archeological Site,35.185,35.185


In [19]:
import pmagpy.controlled_vocabularies3 as cv
reload(cv)
vocab = cv.Vocabulary()
vocabulary, possible_vocabulary = vocab.get_controlled_vocabularies()

-I- Importing controlled vocabularies from https://earthref.org


In [22]:
# check column validity
invalid_cols = [col for col in cols if col not in current_dm.index]


def requiredUnless(col_name, df, arg, *args):
    print 'requiredUnless'
    print 'col_name', col_name
    print 'arg', arg
    print '-'
    return True

def requiredIf(cell_value, *args):
    return True

def requiredIfGroup(cell_value, *args):
    return True


def required(col_name, df, arg):
    print col_name
    #print df
    if col_name in df.columns:
        return True
    else:
        return False
    

def check_max(row, col_name, arg, *args):
    cell_value = row[col_name]
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    arg = float(arg)
    print "{} <= {}".format(cell_value, arg)
    if float(cell_value) <= float(arg):
        return True
    else:
        return "{} must be <= {}".format(str(cell_value), str(arg))

def check_min(row, col_name, arg, *args):
    cell_value = row[col_name]
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    if float(cell_value) >= float(arg):
        return True
    else:
        return "{} must be >= {}".format(str(cell_value), str(arg))

def cv(row, col_name, arg, current_data_model):
    cell_value = row[col_name]
    if cell_value.lower() in vocabulary[arg]:
        return True
    else:
        return "{} is not in {}".format(cell_value, arg)
        

# validate presence
presence_operations = {"required": required, "requiredUnless": requiredUnless}#,
                 #"requiredIf": requiredIf, "requiredIfGroup": requiredIfGroup}
# validate values
value_operations = {"max": check_max, "min": check_min, "cv": cv}

def split_func(string):
    """
    Take a string like 'requiredIf("arg_name")'
    return the function name and the argument:
    (requiredIf, arg_name)
    """
    ind = string.index("(")
    return string[:ind], string[ind+1:-1].strip('"')



for validation_name, validation in current_dm.iterrows():
    val_list = validation['validations']
    if not val_list:
        continue
    for num, val in enumerate(val_list):
        # split val into function & arg
        func_name, arg = split_func(val)
        # first validate for presence
        if func_name in presence_operations:
            func = presence_operations[func_name]
            grade = func(validation_name, current_df, arg)
            pass_col_name = "presence_pass_" + validation_name
            current_df[pass_col_name] = grade

                    
            #continue
        # then validate for correct values
        elif func_name in value_operations:
            func = value_operations[func_name]
            if validation_name in current_df.columns:
                print val
                #grade = current_df[validation_name].apply(func, args=(validation_name, current_df, arg))
                grade = current_df.apply(func, args=(validation_name, arg, current_dm), axis=1)
                current_df["pass_" + validation_name + "_" + str(num)] = grade
        
        #func = operations[func_name]
        #print '-'
        #print 'validating:', validation_name, "with", func
        #print "current_df[{}]".format(validation_name)
        #print current_df[validation_name]
        #grade = current_df[validation_name].apply(func, args=(validation_name, current_df, arg))
        #current_df['pass_{}'.format(validation_name)] = grade
  

# check that values pass validation
# required()
# requiredUnless
# min
# max
# cv (controlled vocab)
# in
# requiredIfGroup
# type
# sv (suggested vocab)



# re-do upload_magic to use contribution-level (??)

# first, do validations on each table in the contribution
# this will include removing unneded data (RmKeys from old upload_magic)
# this will also include checking everything against the data model (strings are strings, etc.)g

# next, do cross-contribution validations (all specimens exist, etc.)

# next, splat out each table into a file and wrap it up.  give it a sensible name.  

pass_cols = current_df.columns.str.match("^pass_")

#current_df[pass_cols]
current_df[current_df.columns[pass_cols]]
#current_dm[['validations']][current_dm[['validations']].index.str.match("lat")]

requiredUnless
col_name age
arg age_low","age_high
-
requiredUnless
col_name age_high
arg age
-
requiredUnless
col_name age_low
arg age
-
age_unit
geologic_classes
min(-90)
max(90)
33.0168 <= 90.0
32.585 <= 90.0
min("lat_s")
lat_n
min(-90)
max(90)
400.0 <= 90.0
32.585 <= 90.0
max("lat_n")
400.0 <= 33.0168
32.585 <= 32.585
lat_s
lithologies
location
cv("location_type")
location_type
min(0)
max(360)
35.568 <= 360.0
35.185 <= 360.0
lon_e
min(0)
max(360)
35.568 <= 360.0
35.185 <= 360.0
lon_w


,pass_lat_n_0,pass_lat_n_1,pass_lat_n_2,pass_lat_s_0,pass_lat_s_1,pass_lat_s_2,pass_location_type_0,pass_lon_e_0,pass_lon_e_1,pass_lon_w_0,pass_lon_w_1
location,,,,,,,,,,,
Tel Hazor,True,True,33.0168 must be >= 400.0,True,400.0 must be <= 90.0,400.0 must be <= 33.0168,Archeological Site is not in location_type,True,True,True,True
Tel Megiddo,True,True,True,True,True,True,Archeological Site is not in location_type,True,True,True,True


In [69]:
current_dm

,description,examples,group,label,notes,position,previous_columns,type,unit,urls,validations
age,Location inferred age,None,Age,Inferred Age,None,38,"[{u'column': u'average_age', u'table': u'pmag_...",Number,Custom,None,"[requiredUnless(""age_low"",""age_high""), require..."
age_high,"Location inferred age, High range",None,Age,Inferred Age High,None,41,"[{u'column': u'average_age_high', u'table': u'...",Number,Custom,None,"[min(""age_low""), requiredUnless(""age"")]"
age_low,"Location inferred age, Low range",None,Age,Inferred Age Low,None,40,"[{u'column': u'average_age_low', u'table': u'p...",Number,Custom,None,"[max(""age_high""), requiredUnless(""age"")]"
age_sigma,"Location inferred age, Uncertainty",None,Age,Inferred Age Sigma,Standard error or standard deviation at one sigma,39,"[{u'column': u'average_age_sigma', u'table': u...",Number,Custom,None,[min(0)]
age_unit,"Location inferred age, Unit",None,Age,Inferred Age Unit,None,42,"[{u'column': u'average_age_unit', u'table': u'...",String,None,None,"[cv(""age_unit""), required()]"
analysts,Colon-delimited list of EarthRef handles and/o...,[@user1:@user2:Not A. Member <no.earthref.hand...,Metadata,Analyst Names,None,97,"[{u'column': u'er_analyst_mail_names', u'table...",List,None,None,"[type(""users"")]"
citations,Colon-delimited list of citation DOIs,"[10.1029/92JB01202, 10.1029/2003GC000635:This ...",Result,Citation Names,None,12,"[{u'column': u'er_citation_names', u'table': u...",List,None,None,"[type(""references""), requiredUnlessTable(""site..."
conglomerate_test,Classification and result of the (intra-format...,None,Direction,Conglomerate Test,None,56,"[{u'column': u'conglomerate_test', u'table': u...",String,Flag,None,"[cv(""conglomerate_test"")]"
contact_test,Classification and result of the (inverse) con...,None,Direction,Baked Contact Test,None,57,"[{u'column': u'contact_test', u'table': u'pmag...",String,Flag,None,"[cv(""contact_test"")]"
continent_ocean,Name for continent or ocean island region,None,Geography,Continent or Ocean Island Region,None,33,"[{u'column': u'continent_ocean', u'table': u'e...",String,None,None,"[cv(""continent_ocean"")]"
